In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
train=pd.read_csv('C:/Users/panig/Downloads/train.csv')
test=pd.read_csv('C:/Users/panig/Downloads/test.csv')

In [ ]:
print(train.shape)
print(test.shape)

Data Cleaning

In [ ]:
test.info(),train.info()

In [ ]:
train.sample(100)

In [ ]:
#Handling null values
#The 'Cabin' column contains numerous null values, so we will drop it.

In [ ]:
train.drop(columns=['Cabin'],inplace=True)
test.drop(columns=['Cabin'],inplace=True)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train['Embarked'].fillna('S',inplace=True)

In [ ]:
test['Fare'].fillna(test['Fare'].mean(), inplace=True)

In [ ]:
#Creating a Unified DataFrame for Easier Manipulation 

In [ ]:
df=pd.concat([train,test],sort=True).reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.corr(numeric_only=True)['Age'].abs()

In [ ]:
df_Age_mean=df.groupby(['Sex', 'Pclass']).median(numeric_only=True)['Age']
df_Age_mean

In [ ]:
df['Age']=df.groupby(['Sex','Pclass'])['Age'].transform(lambda x: x.fillna(x.median()))

In [ ]:
df.isnull().sum()

In [ ]:
#Extracting Titles from Names and Adding as a Separate Column 

In [ ]:
df['Title']=df['Name'].str.split(", ",expand=True)[1].str.split(".",expand=True)[0]

In [ ]:
df['Title'].value_counts()

In [ ]:
df['Title'] = df['Title'].replace(['Lady', 'the Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')

In [ ]:
df['Title'].value_counts()

In [ ]:
#Creating a Family Column by Combining Sibling and Parent Child Column 

In [ ]:
df['Family_size']=df['SibSp'] + df['Parch'] + 1

In [ ]:
df.sample(10)

In [ ]:
df.drop(columns=['Name','Parch','SibSp','Ticket'],inplace=True)

In [ ]:
df.sample(10)

In [ ]:
def family_size(number):
    if number==1:
        return "Alone"
    elif number>1 and number <5:
        return "Small"
    else:
        return "Large"

In [ ]:
df['Family_size']=df['Family_size'].apply(family_size)

Exploratory Data Analysis

In [ ]:
df.info()

In [ ]:
df['Age'] = df['Age'].astype('int64')

In [ ]:
df.info()

In [ ]:
#Finding Survival Correlation with Every Other Column 
#We will analyze the correlation between survival and every other column to understand their relationships.

In [ ]:
col=['Embarked','Pclass','Sex','Title','Family_size']
for i in col:
    print('Survival Correlation by:', i)
    df2=df.groupby(i)['Survived'].mean().reset_index()
    print(df2)
    print('-'*10, '\n')

In [ ]:
#Adding Graphs to Clarify the Picture 
#We will include some visualizations to make the data and its correlations clearer.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


fig = go.Figure()

fig.add_trace(go.Histogram(x=df[df['Survived']==0]['Age'], name='Not Survived', opacity=0.5))
fig.add_trace(go.Histogram(x=df[df['Survived']==1]['Age'], name='Survived', opacity=0.5))

fig.update_layout(
    title='Age Distribution by Survival',
    xaxis_title='Age',
    yaxis_title='Density',
    barmode='overlay',  
    bargap=0.1, 
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=df[df['Survived'] == 0]['Fare'], name='Not Survived', opacity=0.5))
fig.add_trace(go.Histogram(x=df[df['Survived'] == 1]['Fare'], name='Survived', opacity=0.5))

fig.update_layout(
    title='Fare Distribution by Survival',
    xaxis_title='Fare',
    yaxis_title='Density',
    barmode='overlay',  
    bargap=0.1,  
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=df[df['Survived'] == 0]['Family_size'], name='Not Survived', opacity=0.5))
fig.add_trace(go.Histogram(x=df[df['Survived'] == 1]['Family_size'], name='Survived', opacity=0.5))

fig.update_layout(
    title='Family Size Distribution by Survival',
    xaxis_title='Family Size',
    yaxis_title='Density',
    barmode='overlay',  
    bargap=0.1,  
)

fig.show()

In [ ]:
from plotly.subplots import make_subplots
col = ['Embarked', 'Pclass', 'Sex', 'Title']

fig = make_subplots(rows=1, cols=len(col), subplot_titles=col)

for i, c in enumerate(col, start=1):
    grouped_data = df.groupby([c, 'Survived']).size().reset_index(name='Count')
    
    survived_data = grouped_data[grouped_data['Survived'] == 1]
    not_survived_data = grouped_data[grouped_data['Survived'] == 0]
    
    fig.add_trace(go.Bar(x=survived_data[c], y=survived_data['Count'], name='Survived', marker_color='green'), row=1, col=i)
    fig.add_trace(go.Bar(x=not_survived_data[c], y=not_survived_data['Count'], name='Not Survived', marker_color='red'), row=1, col=i)
    
    fig.update_xaxes(title_text=c, row=1, col=i)
    fig.update_yaxes(title_text='Count', row=1, col=i)

# Update figure layout
fig.update_layout(
    title='Countplot of Categorical Variables by Survival',
    height=400,  
    showlegend=True,
    barmode='group',  
)

# Show figure
fig.show()

In [ ]:
sns.pairplot(df,hue='Survived')

In [ ]:
def correlation_heatmap(df):
    _ , ax = plt.subplots(figsize =(14, 12))
    colormap = sns.diverging_palette(220, 10, as_cmap = True)
    
    _ = sns.heatmap(
        df.corr(numeric_only=True), 
        cmap = colormap,
        square=True, 
        cbar_kws={'shrink':.9 }, 
        ax=ax,
        annot=True, 
        linewidths=0.1,vmax=1.0, linecolor='white',
        annot_kws={'fontsize':12 }
    )
    
    plt.title('Pearson Correlation of Features', y=1.05, size=15)

correlation_heatmap(df)

Feature Engineering

In [ ]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn import ensemble
from sklearn import gaussian_process
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn import svm
from sklearn import tree
from sklearn import discriminant_analysis
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
df.head()

In [ ]:
#Separating the Data into Train and Test Sets 
#We will divide the data into training and testing sets to evaluate the model's performance.

In [ ]:
train = df.loc[:890]
test = df.loc[891:]

In [ ]:
test.head()

In [ ]:
test.drop(columns=['Survived'],inplace=True)

In [ ]:
train['Survived'] = train['Survived'].astype('int64')

In [ ]:
train.head()

In [ ]:
train.shape,test.shape

In [ ]:
train = train.drop("PassengerId", axis=1)

In [ ]:
X_train = train.drop("Survived", axis=1)
y_train = train["Survived"]

In [ ]:
X_train.head()

In [ ]:
X_train.shape,y_train.shape

In [ ]:
# Creating a Pipeline with Column Transformer 
#We will build a pipeline that incorporates a Column Transformer to streamline preprocessing and feature engineering.

In [ ]:
num_cat_tranformation=ColumnTransformer([
                                    ('scaling',MinMaxScaler(),[0,2]),
                                    ('onehotencolding1',OneHotEncoder(),[1,3]),
                                    ('ordinal',OrdinalEncoder(),[4]),
                                    ('onehotencolding2',OneHotEncoder(),[5,6])
                                    ],remainder='passthrough')

In [ ]:
bins=ColumnTransformer([
                        ('Kbins',KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile'),[0,2]),
                        ],remainder='passthrough')

In [ ]:
from sklearn import set_config
set_config(display='diagram')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_train,y_train,test_size=0.2)

In [ ]:
def create_pipeline(algo):
    return Pipeline([
        ('num_cat_transformation', num_cat_tranformation),
        ('bins', bins),
        ('classifier', algo)
    ])

Model Training

In [ ]:
#Evaluating Various Machine Learning Models to Find the Best Performer 
#We will test different machine learning models to identify which one provides the best performance for our data.

In [ ]:
algorithms=[
    # Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    # Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    # GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    # Naive Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    # Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    # SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),
    
    # Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    # Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),
    
    # XGBoost
    XGBClassifier()
]

In [ ]:
model_names = []
CV_Accuracy=[]
for algo in algorithms:
    pipeline = create_pipeline(algo)
    scores = cross_val_score(pipeline, X_train, y_train, cv=5)
    model_names.append(algo.__class__.__name__)
    CV_Accuracy.append(scores.mean())
    
#     print(f'Model: {algo.__class__.__name__}, CV Accuracy: {scores.mean()}')
#     print('\n')

In [ ]:
model_df = pd.DataFrame({
    'Model': model_names,
    'Accuracy': CV_Accuracy})

In [ ]:
model_df.sort_values(by='Accuracy', ascending=False, inplace=True)

In [ ]:
#Tuning the Top 3 Machine Learning Models for Better Results 
#We will focus on tuning the top 3 performing models from our data frame to enhance their performance and achieve better results.

In [ ]:
model_df

In [ ]:
algo1 = ensemble.RandomForestClassifier(criterion='gini',
                                           n_estimators=1750,
                                           max_depth=7,
                                           min_samples_split=6,
                                           min_samples_leaf=6,
                                           max_features='auto',
                                           oob_score=True,
                                           random_state=0,
                                           n_jobs=-1,
                                           verbose=1) 

algo2 =ensemble.GradientBoostingClassifier()
algo3=linear_model.RidgeClassifierCV()

In [ ]:
pipe1=Pipeline([
        ('num_cat_transformation', num_cat_tranformation),
        ('bins', bins),
        ('classifier', algo2)
    ])

In [ ]:
pipe1.fit(X_train,y_train)

In [ ]:
y_pred = pipe1.predict(X_test)
accuracy_score(y_test,y_pred)

In [ ]:
submission = pd.DataFrame()
submission["PassengerId"] = test["PassengerId"]
submission

In [ ]:
test = test.drop("PassengerId", axis=1)
test.head()

In [ ]:
predictions = pipe1.predict(test)
submission["Survived"] =  predictions
submission.to_csv("submission.csv", index=False)

WE HAVE THE ACCURACY OF 87% WHICH IS QUITE GOOD AND THE MODEL CAN PREDICT THE DATA QUITE ACCURATELY.